## 数据合成 (simplified)

参考 [合成数据: 利用开源技术节约资金、时间和减少碳排放](https://github.com/huggingface/blog/blob/main/zh/synthetic-data-save-costs.md) 一文，把对应的模型换成SiliconFlow用户常用的 Qwen2.5-72B-Instruct 进行数据合成。

### 步骤1. 安装并import对应库

In [73]:
# install requirements
# %%bash
%pip install --upgrade pip -q
%pip install datasets~=2.16.1
%pip install openai
%pip install scikit-learn
%pip install pandas
%pip install tqdm
%pip install python-dotenv

8194.19s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


8200.59s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


8206.58s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...

Note: you may need to restart the kernel to use updated packages.


8224.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


8229.83s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [74]:
import os
from tqdm import tqdm
import ast
import numpy as np
import pandas as pd
import random
import json
from datetime import datetime
import os
import requests
from datasets import load_dataset
import random
from dotenv import load_dotenv
from openai import OpenAI

### 步骤 2. 设置对应环境变量

In [75]:
load_dotenv() 

# 获取环境变量
API_KEY = os.getenv('SILICONFLOW_API_KEY')

if API_KEY is None:
    print("API_KEY not found in environment variables")
else:
    print("API_KEY loaded successfully")

client = OpenAI(
    api_key=API_KEY, # 从https://cloud.siliconflow.cn/account/ak获取
    base_url="https://api.siliconflow.cn/v1" # 此处使用硅基流动Qwen2.5-72B-Instruct来进行数据处理
)

# choose one of these API providers: "SF" or "OAI"
API_PROVIDER = "SF"

MODEL_NAME = "Qwen/Qwen2.5-72B-Instruct"

API_KEY loaded successfully


In [76]:

SEED = 42
N_SAMPLE = False  # You can sample parts of the data for faster testing. False for run on full dataset, int for sampling
SELF_CONSISTENCY_ITERATIONS = 3  # How many times should the model try to predict the same text for self-consistency?
DATA_SUBSET = "sentences_allagree"  # "sentences_allagree", "sentences_66agree", "sentences_75agree"

### 步骤 3. 处理数据集

In [77]:
# financial_phrasebank paper: https://arxiv.org/pdf/1307.5336.pdf
random.seed(SEED)

# load dataset
dataset = load_dataset(
    "financial_phrasebank", DATA_SUBSET, 
    split="train"  # note that the dataset does not have a default test split
)

# sample for faster testing
if N_SAMPLE: 
    dataset = dataset.select(random.sample(range(len(dataset)), N_SAMPLE))


/Users/senseb/miniconda3/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [78]:
# create a new column with the numeric label verbalised as label_text (e.g. "positive" instead of "0")
label_map = {i: label_text for i, label_text in enumerate(dataset.features["label"].names)}

def add_label_text(example):
    example["label_text"] = label_map[example["label"]]
    return example

dataset = dataset.map(add_label_text)[:10]

print(dataset)

{'sentence': ['According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .', "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .", 'In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .', 'Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .', 'Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .', 'Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .', "Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17

### 设置对应 Prompt

In [79]:
# prompt is inspired by the annotator instructions provided in section "Annotation task and instructions"
# in the financial_phrasebank paper: https://arxiv.org/pdf/1307.5336.pdf

prompt_financial_sentiment = """\
You are a highly qualified expert trained to annotate machine learning training data.

Your task is to analyze the sentiment in the TEXT below from an investor perspective and label it with only one the three labels:
positive, negative, or neutral.

Base your label decision only on the TEXT and do not speculate e.g. based on prior knowledge about a company. 

Do not provide any explanations and only respond with one of the labels as one word: negative, positive, or neutral

Examples:
Text: Operating profit increased, from EUR 7m to 9m compared to the previous reporting period.
Label: positive
Text: The company generated net sales of 11.3 million euro this year.
Label: neutral
Text: Profit before taxes decreased to EUR 14m, compared to EUR 19m in the previous period.	
Label: negative

Your TEXT to analyse:
TEXT: {text}
Label: """



prompt_financial_sentiment_cot = """\
You are a highly qualified expert trained to annotate machine learning training data.

Your task is to briefly analyze the sentiment in the TEXT below from an investor perspective and then label it with only one the three labels:
positive, negative, neutral.

Base your label decision only on the TEXT and do not speculate e.g. based on prior knowledge about a company. 

You first reason step by step about the correct label and then return your label.

You ALWAYS respond only in the following JSON format: {{"reason": "...", "label": "..."}}
You only respond with one single JSON response. 

Examples:
Text: Operating profit increased, from EUR 7m to 9m compared to the previous reporting period.
JSON response: {{"reason": "An increase in operating profit is positive for investors", "label": "positive"}}
Text: The company generated net sales of 11.3 million euro this year.
JSON response: {{"reason": "The text only mentions financials without indication if they are better or worse than before", "label": "neutral"}}
Text: Profit before taxes decreased to EUR 14m, compared to EUR 19m in the previous period.	
JSON response: {{"reason": "A decrease in profit is negative for investors", "label": "negative"}}

Your TEXT to analyse:
TEXT: {text}
JSON response: """



### 4. 测试数据处理

In [80]:
labels = ["positive", "negative", "neutral"]

def clean_output(string, random_choice=True):
    for category in labels:
        if category.lower() in string.lower():
            return category
    # if the output string cannot be mapped to one of the categories, we either return "FAIL" or choose a random label
    if random_choice:
        return random.choice(labels)
    else:
        return "FAIL"


def process_output_cot(output):
    try: 
        output_dic = ast.literal_eval(output) 
        return output_dic
    except Exception as e:
        # if json/dict parse fails, do simple search for occurance of first label term
        print(f"Parsing failed for output: {output}, Error: {e}")
        output_cl = clean_output(output, random_choice=False)
        output_dic = {"reason": "FAIL", "label": output_cl}
        return output_dic


generation_params = dict(
    top_p=0.90,
    temperature=0.8,
    max_tokens=128,
)

def generate_text(prompt=None, generation_params=None):
    response = client.chat.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        **generation_params
    )

    return response.choices[0].message.content


In [72]:

output_simple = []
for text in tqdm(dataset["sentence"]):
    output = generate_text(
        prompt=prompt_financial_sentiment.format(text=text), generation_params=generation_params
    )
	# clean output
    output_cl = clean_output(output, random_choice=True)
    output_simple.append(output_cl)
    
    
SELF_CONSISTENCY_ITERATIONS = 3

output_cot_multiple = []
for _ in range(SELF_CONSISTENCY_ITERATIONS):
    output_lst_step = []
    for text in tqdm(dataset["sentence"]):
        prompt_formatted = prompt_financial_sentiment_cot.format(text=text)
        output = generate_text(
            prompt=prompt_financial_sentiment_cot.format(text=text), generation_params=generation_params
        )
        output_dic = process_output_cot(output)
        output_lst_step.append(output_dic["label"])

    output_cot_multiple.append(output_lst_step)


100%|██████████| 10/10 [00:22<00:00,  2.25s/it]


In [81]:
from collections import Counter

def find_majority(row):
    # Count occurrences
    count = Counter(row)
    # Find majority
    majority = count.most_common(1)[0]
    # Check if it's a real majority or if all labels are equally frequent
    if majority[1] > 1:
        return majority[0]
    else: # in case all labels appear with equal frequency
        return random.choice(labels)

df_output = pd.DataFrame(data=output_cot_multiple).T

df_output['label_pred_cot_multiple'] = df_output.apply(find_majority, axis=1)
df_output

,0,1,2,label_pred_cot_multiple
0,neutral,neutral,neutral,neutral
1,positive,positive,positive,positive
2,positive,positive,positive,positive
3,positive,positive,positive,positive
4,positive,positive,positive,positive
5,positive,positive,positive,positive
6,positive,positive,positive,positive
7,positive,positive,positive,positive
8,positive,positive,positive,positive
9,positive,positive,positive,positive


In [82]:
from sklearn.metrics import classification_report

def compute_metrics(label_experts, label_pred):
		# classification report gives us both aggregate and per-class metrics 
    metrics_report = classification_report(
        label_experts, label_pred, digits=2, output_dict=True, zero_division='warn'
    )

    return metrics_report

label_experts = dataset["label_text"]
label_pred = output_simple
label_pred_cot_multiple = df_output['label_pred_cot_multiple']

print(label_experts)
print(label_pred)

metrics_simple = compute_metrics(label_experts, label_pred)
metrics_cot_multiple = compute_metrics(label_experts, label_pred_cot_multiple)


['neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive']
['neutral', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive', 'positive']


In [83]:

df_train = pd.DataFrame({
    "text": dataset["sentence"],
    "labels": df_output['label_pred_cot_multiple']
})

df_train.to_csv("df_train.csv")


In [84]:
df_train.head(10)

,text,labels
0,"According to Gran , the company has no plans t...",neutral
1,"For the last quarter of 2010 , Componenta 's n...",positive
2,"In the third quarter of 2010 , net sales incre...",positive
3,Operating profit rose to EUR 13.1 mn from EUR ...,positive
4,"Operating profit totalled EUR 21.1 mn , up fro...",positive
5,Finnish Talentum reports its operating profit ...,positive
6,Clothing retail chain Sepp+ñl+ñ 's sales incre...,positive
7,Consolidated net sales increased 16 % to reach...,positive
8,Foundries division reports its sales increased...,positive
9,"HELSINKI ( AFX ) - Shares closed higher , led ...",positive


### 5. 评估价格对比 

In [ ]:
#### per-token costs siliconflow
